# Part II: Spark SQL and DataFrames

Author: **Julien Peloton** [@JulienPeloton](https://github.com/astrolabsoftware/spark-tutorials/issues/new?body=@JulienPeloton)  
Last Verifed to Run: **2018-10-29**  

__Learning objectives__

- Apache Spark Data Sources.
- Loading and distributing data: Spark SQL and DataFrames.
- Hands-on: RDD vs DataFrame, partitioning, limits, ...

## Apache Spark Data Sources

### A tour of data formats

There are many data formats used in the context of Big Data: CSV (1978), XML (1996), JSON (2001), Thrift (2007), Protobuf (2008), Avro & SequenceFile (2009), Parquet (2013), ORC (2016), and the list goes on... Some are _naively_ structured that is using a single type to describe the data (e.g. text) without any internal organisation to access faster the data. Others are more complex and highly optimised for big data treatment (e.g. Parquet). Unfortunately those are not the data formats typically chosen by the scientific community. In astronomy you would rather store the data in FITS (1981) or HDF5 (1988) format. 
FITS and HDF5 are multi-purposes data formats: images, spectra, photon lists, data cubes, or even structured data such as multi-table databases can be efficiently stored and accessed.

### Connecting to Data Source

The data source API in Apache Spark belongs to the [Spark SQL module](https://spark.apache.org/sql/). Note that Spark Core has some simple built-in ways to read data from disk (binary or text), but Spark SQL is more complete. If you want to connect a particular data source with Apache Spark, you have mostly two ways:

- [indirect] Access and distribute your files as binary streams (Spark does it natively), and decode the data on-the-fly within executors using third-party libraries.
- [native] Write a custom connector to access, distribute and decode the data natively.

FITS or HDF5 as most of scientific data formats, were not designed for serialisation (distribution of data over machines) originally and they often use compression to reduce the size on disk. Needless to say that default Spark cannot read those natively. 

First attempts to connect those data formats (see e.g. [1] for FITS) with Spark were using the indirect method above. By reading files as binary streams, the indirect method has the advantage of having access to all FITS functionalities implemented in the underlying user library. This can be an advantage when working with the Python API for example which already contains many great scientific libraries. However this indirect method assumes each Spark mapper will receive and handle one entire file (since the filenames are parallelized and entire file data must be reconstructed from binary once the file has been opened by a Spark mapper). Therefore each single file must fit within the memory of a Spark mapper, hence the indirect method cannot distribute a dataset made of large FITS files (e.g. in [1] they have a 65 GB dataset made of 11,150 files). In addition by assuming each Spark mapper will receive and handle one entire file, the indirect method will have a poor load balancing if the dataset is made of files with not all the same size.

Fortunately Apache Spark low-level layers are sufficiently well written to allow extending the framework and write native connectors for any kind of data sources. Recently connectors for FITS and HDF5 were made available [2, 3] to the community. With such connectors, there is a guarantee of having a good load balancing regardless the structure of the dataset and the size of the input files is no more a problem (a 1 TB dataset made of thousand 1 GB files or one single 1 TB file will be viewed as almost the same by a native Spark connector). Note however that the Data Source API is in Java/Scala and if there is no library to play with your data source in those languages you must implement it (what has been done in [2]) or interface with another language.

Note that the low-level layers dealing with the data sources have been recently updated. Apache Spark 2.3 introduced the Data Source API version 2. While the version 1 is still available and usable for a long time, we expect that all Spark connectors will comply with this v2 in the future.

[1] Z. Zhang and K. Barbary and F. A. Nothaft and E. R. Sparks and O. Zahn and M. J. Franklin and D. A. Patterson and S. Perlmutter, Kira: Processing Astronomy Imagery Using Big Data Technology, DOI 10.1109/TBDATA.2016.2599926.  
[2] Peloton, Julien and Arnault, Christian and Plaszczynski, Stéphane, FITS Data Source for Apache Spark, Computing and Software for Big Science (1804.07501). https://github.com/astrolabsoftware/spark-fits   
[3] Liu, Jialin and Racah, Evan and Koziol, Quincey and Canon, Richard Shane, H5spark: bridging the I/O gap between Spark and scientific data formats on HPC systems, Cray user group (2016). https://github.com/valiantljk/h5spark  

## Loading and distributing data: Spark SQL and DataFrames

### DataFrameReader

The interface to read data from disk is always the same for any kind of built-in and officially supported data format:

```python
df = spark.read\
    .format(format: str)\
    .option(key: str, value: Any)\
    # ...
    .option(key: str, value: Any)\
    .load(path: str)
```
 
Note that for most of the data sources, you can use wrappers such as:

```python
spark.read.csv(path, key1=value1, key2=value2, ...)
```

**Format**: The format can be "csv", "json", "parquet", etc. 

**Options**: The number of options depends on the underlying data source. Each has its own set of options. 
In most of the case, no options are needed, but you might want to explore the different possibilities at some point. Surprisingly it is not easy to find documentation and the best remains to read the source code documentation. In pyspark you can easily access it via the wrappers (use IPython for the driver -- see part 1):

```python
# DataFrameReader object
df_reader = spark.read

# Doc on reading CSV
df_reader.csv?
# doc printed

# Doc on reading Parquet
df_reader.parquet?
# doc printed
```


**Path**: The way to specify path is threefold: either a single file (`path/to/folder/myfile.source`), or an entire folder (`path/to/folder`), or a glob pattern (`path/to/folder/*pattern*.source`). Note that you also need to specify the type of file system you are using. Example:

``` python
# Connect to hdfs
path = 'hdfs:///path/to/data'

# Connect to S3
path = 's3:///path/to/data'

# Connect to local file system
path = 'files:///path/to/data'
```




### Using a custom connector

You can also connect to custom connector not included in the default Spark distribution. To do so, you will need to specify the dependencies when submitting your job or invoking your shell (see part 1). If your connector is available through [Maven Central Repository](https://search.maven.org/), you can easily specify it via:

```
# Direct download from central repository
spark-submit --packages groupId:artifactId:version ...
```

Note that this is the same syntax when launching the `pyspark` shell.
For example, if you want to read FITS files using the [spark-fits](https://github.com/astrolabsoftware/spark-fits) connector you would add the following:

```
# Direct download from central repository
spark-submit --packages com.github.astrolabsoftware:spark-fits_2.11:0.7.1 ...
```

You can find the spark-fits entry in the Maven Central [here](https://search.maven.org/artifact/com.github.astrolabsoftware/spark-fits_2.11/0.7.1/jar) for reference.
Alternatively you can download the source code for a particular connector, compile it and include the `jars`:

```
# Specify manually the dependency
spark-submit --jars /path/to/lib/spark-fits.jars ...
```

Note that when you launch `pyspark`, already a numbers of `jars` are included by default (the ones for Spark for example).

## Hands-on

You will find test data in the folder `spark-tutorials/data`.

In [1]:
from pyspark.sql import SparkSession

# Initialise our Spark session
spark = SparkSession.builder.getOrCreate()

### Loading Data: simply structured data (text)

In [104]:
# Load simple CSV file
df = spark.read.format("csv")\
    .load("data/simple.csv")
df.printSchema()

root
 |-- _c0: string (nullable = true)
 |-- _c1: string (nullable = true)



Notice by default the CSV connector interpret all entries as String, and give dummy names to columns. You can infer the data type and use the first row as column names by specifying options:

In [55]:
df = spark.read.format("csv")\
    .option('inferSchema', True)\
    .option('header', True)\
    .load("data/simple.csv")
df.printSchema()

root
 |-- name: string (nullable = true)
 |-- age: integer (nullable = true)



In [56]:
# Make a nice representation of our data
df.show()

+----+---+
|name|age|
+----+---+
|toto|  3|
|tutu|  4|
|titi|  1|
+----+---+



Note that you could do the same with the RDD API, but you would need more steps:

In [97]:
# Load data using Spark RDD API
rdd = sc.textFile("data/simple.csv")

In [66]:
# Each row is a String with all 
# column value
print("First two elements of the RDD are: {}\n".format(rdd.take(2)))

First two elements of the RDD are: ['name,age', 'toto,3']



In [100]:
# First row contains column names
names = rdd.map(lambda x: x.split(",")).take(1)[0]

# Make it a DataFrame
# 1. remove the first line which contains metadata
# 2. split row elements into columns
# 3. Assign correct data type
# 4. Promote to DataFrame
df = rdd\
    .filter(lambda x: 'name' not in x)\
    .map(lambda x: x.split(","))\
    .map(lambda x: [x[0], int(x[1])])\
    .toDF(names)
df.printSchema()

root
 |-- name: string (nullable = true)
 |-- age: long (nullable = true)



### Loading Data: complex structured data (Parquet)

More complex data format can infer automatically schema, and data types.
They are also optimised for fast data access and small memory consumption.

In [83]:
# Same using Parquet - Note that the schema and the data types 
# are directly inferred. (use long for int by default).
df_parquet = spark.read.format("parquet").load("data/simple.parquet")
df_parquet.printSchema()
df_parquet.show()

root
 |-- name: string (nullable = true)
 |-- age: long (nullable = true)

+----+---+
|name|age|
+----+---+
|toto|  3|
|tutu|  4|
|titi|  1|
+----+---+



### DataFrame to RDD

Note that you can always go from a DataFrame back to the underlying RDD:

In [116]:
rdd_from_df = df_parquet.rdd
rdd_from_df.collect()

[Row(name='toto', age=3), Row(name='tutu', age=4), Row(name='titi', age=1)]

Notice that the returned type is a list of `Row`. `Row` is an internal Spark type. You can easily convert it to `dict`, `list` or `numpy.ndarray` at your convenience.

In [115]:
arow = rdd_from_df.take(1)[0]

# Conversion to dict
adict = arow.asDict()
print(type(adict), adict)

# Conversion to list
alist = list(arow)
print(type(alist), alist)

# Conversion to numpy.ndarray
# Data types are not preserved by default
import numpy as np
anarray = np.array(arow)
print(type(anarray), anarray)

<class 'dict'> {'name': 'toto', 'age': 3}
<class 'list'> ['toto', 3]
<class 'numpy.ndarray'> ['toto' '3']


### Partitioning

Partition size, resources, etc...

### Parallelizing existing collection: beware of the 2G points per partition!

In practice, there are two ways to create RDDs: referencing a dataset in an external storage system (see above), or parallelizing an existing collection in your driver program:

In [108]:
mylist = range(100)
rdd_collection = sc.parallelize(mylist)

While it seems a good idea sometimes, you need to be aware that for historical reasons there is a limit on how many numbers can be stored inside one partition: `2**31 - 1`! This comes from the fact that Java Arrays are indexed by int values. Accessing array component beyond this limit will result in a compile-time error.

## Going further

Here is a series of useful links on similar topics:

- Spark SQL module: https://spark.apache.org/sql/
- Spark SQL code on GitHub: https://github.com/apache/spark/tree/master/sql
- Spark SQL doc: http://spark.apache.org/docs/latest/sql-programming-guide.html
- Databricks Data Source documentation: https://docs.databricks.com/spark/latest/data-sources/index.html
- Apache Spark Data Source V2 explained in video (Spark Summit 2018): https://databricks.com/session/apache-spark-data-source-v2
- Introducing Apache Spark Data Sources API V2 (IBM): https://developer.ibm.com/code/2018/04/16/introducing-apache-spark-data-sources-api-v2/